### Portfolio Analysis based on your Risk Aversion level and timeframe
We have 3 levels of Risk Aversion, with level 3 being the most aggressive
We have 3 timeframes for which we will do our Monte Carlo forecasts: 5, 10, and 20 years.

In [12]:
# Import the required libraries and dependencies
#import questionary   #Tried to include this and didn't work
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [13]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [15]:
#Get user input for risk tolerance. User must input 1, 2, or 3. If risk_tolerance is not one of these, we prompt user
#to try again.
risk_tolerance = input("Please select your risk tolerance level from 1-3, 3 being the most aggressive")
#print(risk_tolerance)
#type if statment here
if risk_tolerance != 3 and risk_tolerance != 2 and risk_tolerance != 1:
    risk_tolerance = input("Please select your risk tolerance level from 1-3, 3 being the most aggressive. You must enter 1, 2, or 3.")
    #print(risk_tolerance)
else:
    print("Thank you for your response. Now for the last question.")

print(risk_tolerance)

    

Please select your risk tolerance level from 1-3, 3 being the most aggressive hello
Please select your risk tolerance level from 1-3, 3 being the most aggressive. You must enter 1, 2, or 3. 2


2


In [16]:
time_horizon = input("Please select your time horizon for the forecast: 5, 10, or 20 years?")
print(time_horizon)

#type if statement here if time_horizon is not 5, 10, or 20 we prompt the user to try again

Please select your time horizon for the forecast: 5, 10, or 20 years? 5


5


In [ ]:
# We'll assume 100 shares in each stock of interest

#Briggs let's make this part of what stocks to use dynamic depending on if client selects 
#level 1 2 3 risk tolerance.  Since this is Nev's set we'll assume this is Level 2 for now.
fb_shares = 100  #FB
aapl_shares = 100 #AAPL
tsla_shares = 100 #TSLA
rdsa_shares = 100 #RDS.A  shell oil
gs_shares = 100 #GS


In [ ]:
# Set the variables for the Alpaca API and secret keys. Will be making calls for historical prices.
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")


In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio

#This setting of tickers will also need to be made a bit more dynamic again dep on what client selects
#We know this would be risk tolerance level 2 based on Nev's list.
tickers = ["FB","AAPL","TSLA","RDS.A","GS"]

# Set timeframe to 1D 
timeframe = "1D"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2021-10-27", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-27", tz="America/New_York").isoformat()
limit_rows = 1000
#We can possibly make this start and end date a bit more dynamic by just pulling today's or yesterday's
#date and feeding in here as a variable.

In [ ]:
# We use the Alpaca get_barset function to get current closing prices for our portfolio of 5 tickers
prices_df = alpaca.get_barset(tickers, timeframe, start=start_date, end=end_date, limit=limit_rows).df

# checking dataframe to make sure everything looks ok for our closes
prices_df.head()


In [ ]:
# Access the closing prices for our 5 tickers from the Alpaca DataFrame
# We also convert the value to a floating point number
#Again will need to be made dynamic depending on if level 1 2 or 3 risk tolerance level is selected
fb_close_price = float(prices_df["FB"]["close"])
aapl_close_price = float(prices_df["AAPL"]["close"])
tsla_close_price = float(prices_df["TSLA"]["close"])
rdsa_close_price = float(prices_df["RDS.A"]["close"])
gs_close_price = float(prices_df["GS"]["close"])

# Print the AGG closing price
display(fb_close_price)
display(aapl_close_price)
display(tsla_close_price)
display(rdsa_close_price)
display(gs_close_price)


In [ ]:
# Calculate the current value of the bond portion of the portfolio
#Again will need to be made dynamic depending on if level 1 2 or 3 risk tolerance level is selected
fb_value = fb_shares * fb_close_price
aapl_value = aapl_shares * aapl_close_price
tsla_value = tsla_shares * tsla_close_price
rdsa_value = rdsa_shares * rdsa_close_price
gs_value = gs_shares * gs_close_price

# Print the current value of the bond portfolio
print(f"The current value of your Facebook holding is ${fb_value:.2f}")
print(f"The current value of your Apple holding is ${aapl_value:.2f}")
print(f"The current value of your Tesla holding is ${tsla_value:.2f}")
print(f"The current value of your Shell holding is ${rdsa_value:.2f}")
print(f"The current value of your Goldman Sachs holding is ${gs_value:.2f}")


In [ ]:
# Calculate the total value of the entire portfolio
portfolio_value =  fb_value + aapl_value + tsla_value + rdsa_value + gs_value

# Print the current balance of the stock and bond portion of the portfolio
print(f"The current value of your total portfolio holding is ${portfolio_value:.2f}")



In [ ]:
# Set start and end dates of 10 years back from your current date
#We'll always assume 10 years back. Again we can make this dynamic if we want
#and start with yesterday's date and then go back 10 years.
start_date = pd.Timestamp("2011-10-28", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-28", tz="America/New_York").isoformat()

limit_rows = 1000

In [ ]:
# Use the Alpaca get_barset function to make the API call to get the 3 years worth of pricing data
# The tickers and timeframe parameters should have been set in Part 1 of this activity 
# The start and end dates should be updated with the information set above
# Remember to add the df property to the end of the call so the response is returned as a DataFrame
prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=limit_rows
).df

# Display both the first and last five rows of the DataFrame
display(prices_df.head(2))
display(prices_df.tail(2))
tickers

###Here we run a Monte Carlo simulation of 1000 samples and the time frame could be 5, 10, or 20 years depending on the user's response. We always will assume equal 20% weight for each stock in the portfolio.

In [ ]:
# Configure the Monte Carlo simulation to forecast cumulative returns based on the client's input
# The weights we will assume will be split evenly 20 percent for each ticker.
# We will be running 1000 samples.
#This part will only change based on if client selects 5 10 or 20 years to do the forecast.
MC_20All = MCSimulation(
    portfolio_data = prices_df,
    weights = [0.2,0.2,0.2,0.2,0.2],
    num_simulation = 1000,
    num_trading_days = 252*10  #This number 10 will need to be made a variable based off client's input
)

# Review the simulation input data
MC_20All.portfolio_data.head()


In [ ]:
# Run the Monte Carlo simulation to forecast client specified number of years of cumulative returns
MC_20All.calc_cumulative_return()

#Tweaked monte carlo py file to spit out a message every 200 simulations.

In [ ]:
# Visualize the Monte Carlo simulation by creating an overlay line plot
weight_20All_plot = MC_20All.plot_simulation()


In [ ]:
# Here we visualize the probability distribution of the Monte Carlo simulation 
# based on the user's timeframe by plotting a histogram
distribution_20All_plot = MC_20All.plot_distribution()


#### Step 4: Generate the summary statistics for the Monte Carlo simulation.

In [ ]:
# Generate summary statistics from the Monte Carlo simulation results
# We save the results as a variable
weight_20All_table = MC_20All.summarize_cumulative_return()


# Review the Client's Monte Carlo summary statistics
print(weight_20All_table)

In [ ]:
# We re-print the current balance of the total portfolio of the client
print(f"The current value of your total portfolio holding is ${portfolio_value:.2f}")

#Will leave this up to the group. We can just say hey this is the portfolio holding value
#or we can also break it down by individual stock holding like we did above.

print("______________________________________")


In [ ]:
# We now use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes 
#for the current client portfolio
ci_lower_thirty_cumulative_return = round(weight_20All_table[8]*portfolio_value,2)
ci_upper_thirty_cumulative_return = round(weight_20All_table[9]*portfolio_value,2)

# Print the result of your calculations
print(f"There is a 95% chance that the current value of ${portfolio_value:.2f} in the portfolio with an "
      f"even 20% weight in each stock over the next 10 years will end within the range of"
      f" ${ci_lower_thirty_cumulative_return} and ${ci_upper_thirty_cumulative_return}.")
